# Implémentation de l'architecture des réseaux de neurones

*http://exo7.emath.fr/cours/livre-deepmath.pdf* 

## Chargement des modules et données

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import tensorflow.keras

In [3]:
df = pd.read_csv('C:/Users/mathilde.binet/Desktop/PIF/données_bloom_5000.csv',sep = ';')

#dataset des 5.000 premiers bonds récupérés de bloom

In [4]:
df.head()

,Issuer Name,Cpn,Maturity,Issue Price,ISIN,Issue Date,Par Amt,Currency,DES Notes,Ticker,Debt / Common Equity,Debt/EBITDA,EBITDA
0,European Union,0.0,07/04/2035,101.500,EU000A285VM2,12/01/2020,1000.000,EUR,NaN,EU,NaN,NaN,NaN
1,HSBC Holdings PLC,4.6,NaN,100.000,US404280CN71,12/17/2020,815.641,USD,PAR CALL 6 MONTHS PRIOR TO EACH RESET DATE.,HSBC,255.2640,NaN,NaN
2,CNAC HK Finbridge Co Ltd,3.0,09/22/2030,99.863,XS2226808165,09/22/2020,854.394,USD,MANDATORY NON-REGISTERATION CALL@100%,HAOHUA,785.8010,11.3875,23293.5
3,Deutsche Lufthansa AG,3.0,05/29/2026,99.390,XS2265369657,12/01/2020,100000.000,EUR,NaN,LHAGR,99.5959,NaN,4552.0
4,Credit Suisse Group AG,4.5,NaN,100.000,USH3698DDD33,12/09/2020,828.332,USD,NaN,CS,630.2450,NaN,NaN


In [5]:
df.shape

(5000, 13)

In [6]:
#RECUPERER VRAIES DONNEES DE PROBA DE DEFAUT ARGGGHHHHHHHHHHHHHHHHHHHHHHHHH
#à partir du premier algo, estimer PD? données à récupérer, notamment le PRIXXXX 

#j'ai mis des valeurs aléatoires pour créer la colonne

#df["PD"] = pd.Series(np.random.random(len(df["ISIN"])), index=df.index)

In [7]:
#mettre les ratios financiers
df1 = df[["PD","Debt / Common Equity","Debt/EBITDA","EBITDA"]]

In [8]:
df1 = df1.dropna(how = 'any') 

#on peut pas faire tourner l'algo avec des valeurs manquantes. Voir techniques pour les remplacer.

## Traitement des données 

In [9]:
#Préparation bases de train et de test 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df1.drop("PD",axis=1),df1["PD"],test_size=0.2, random_state=42) 

In [10]:
#Standardisation des données 

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=True,with_std=True)
scaler.fit(X_train)

x_train = scaler.transform(X_train)
x_train = pd.DataFrame(x_train, index=X_train.index, columns=X_train.columns)

x_test = scaler.transform(X_test)
x_test = pd.DataFrame(x_test, index=X_test.index, columns=X_test.columns)

In [11]:
x_test.head()

,Debt / Common Equity,Debt/EBITDA,EBITDA
3787,-0.024283,-0.022071,0.548917
829,-0.024148,-0.017536,-0.076796
2541,-0.024027,-0.022374,-0.074564
4732,-0.024106,-0.022412,-0.049289
1871,-0.024129,-0.022251,-0.077153


## Réseaux de neurones 

In [21]:
#RESEAUX DE NEURONES 
#http://eric.univ-lyon2.fr/~ricco/tanagra/fichiers/fr_Tanagra_Packages_Python_for_Deep_Learning.pdf

In [ ]:
#Explications très bien dans ce document : 

#http://exo7.emath.fr/cours/livre-deepmath.pdf 

#très bon document (parties Python, keras avec les codes et explications)

In [12]:
#Importation des classes
from keras.models import Sequential
from keras.layers import Dense

#Architecture du réseau
modele = Sequential()

#Couche de neurones 

#première couche
modele.add(Dense(units=2,input_dim=3,activation="sigmoid"))
#2: nombre de neurones à la première couche, 3: nombre d'inputs

#couche cachée vers sortie
modele.add(Dense(units=1,activation="sigmoid"))
#1: 1 sortie, nb d'inputs = nb d'output de la couche 1 (3)

J'ai pris la fonction sigmoïde car elle est continue et à valeurs dans [0,1] (comme les probabilités de défaut). Régression logit. 

In [13]:
#Choix de la méthode de descente de gradient

from keras.optimizers import SGD
learning_rate = 0.5

sgd = SGD(learning_rate)

In [14]:
modele.compile(loss='mean_squared_error',optimizer=sgd,metrics=['accuracy'])

In [15]:
print(modele.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 8         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
#Calcul des poids par descente de gradient 

modele.fit(x_train,y_train, batch_size = 500, epochs = 40)

Epoch 1/40
5/5 [==============================] - 13s 2ms/step - loss: 0.0899 - accuracy: 0.0000e+00
Epoch 2/40
5/5 [==============================] - 0s 2ms/step - loss: 0.0863 - accuracy: 0.0000e+00
Epoch 3/40
5/5 [==============================] - 0s 3ms/step - loss: 0.0869 - accuracy: 0.0000e+00
Epoch 4/40
5/5 [==============================] - 0s 2ms/step - loss: 0.0854 - accuracy: 0.0000e+00
Epoch 5/40
5/5 [==============================] - 0s 2ms/step - loss: 0.0854 - accuracy: 0.0000e+00
Epoch 6/40
5/5 [==============================] - 0s 4ms/step - loss: 0.0868 - accuracy: 0.0000e+00
Epoch 7/40
5/5 [==============================] - 0s 2ms/step - loss: 0.0859 - accuracy: 0.0000e+00
Epoch 8/40
5/5 [==============================] - 0s 2ms/step - loss: 0.0860 - accuracy: 0.0000e+00
Epoch 9/40
5/5 [==============================] - 0s 2ms/step - loss: 0.0861 - accuracy: 0.0000e+00
Epoch 10/40
5/5 [==============================] - 0s 2ms/step - loss: 0.0859 - accuracy: 0.0000e+0

In [17]:
#Résultats 

resultat = modele.evaluate(x_test, y_test, verbose=0)
print("Valeur de l'erreur sur les données de test (loss):", resultat[0])
print('Précision sur les données de test (accuracy):', resultat[1])

Valeur de l'erreur sur les données de test (loss): 0.08323077112436295
Précision sur les données de test (accuracy): 0.0
